In [1]:
from utils import *
from itertools import chain
import nonlinshrink as nls
from sklearn.covariance import LedoitWolf
from sklearn.covariance import OAS
from scipy.stats import moment

# Vectorized Backtesting

In [2]:
class GMVP_backtesting(vectorized_backtesting):
    def get_portfolio(self, year):
        portfolio = self.get_GMVP(volatility_vector = self.volatility_aggregate.loc[year - 1], 
                            correlation_matrix = self.correlation_aggregate.loc[year - 1])
        return portfolio


In [3]:
GMVP_backtest = GMVP_backtesting()

In [4]:
test = GMVP_backtest.get_portfolio_daily_return(1999, 2000)

In [5]:
len(test)

504

In [ ]:
def get_shrink_portfolio()

In [20]:
covariance_shrunk = LedoitWolf().fit(stock_returns.loc[i]).covariance_

# Backtrader Test

In [1]:
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)

import datetime  # For datetime objects
import os.path  # To manage paths
import sys  # To find out the script name (in argv[0])

# Import the backtrader platform
import backtrader as bt


In [61]:

# Create a Stratey
class TestStrategy(bt.Strategy):

    def log(self, txt, dt=None):
        ''' Logging function fot this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.spy = self.datas[0].close
        self.amzn = self.datas[1].close

        # To keep track of pending orders and buy price/commission
        self.order = None
        self.buyprice = None
        self.buycomm = None

        # time
        # self.month = self.datas[0].datetime.month

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))

                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))

    def next(self):
        month_current = self.datas[0].datetime.date(0).month
        month_next = self.datas[0].datetime.date(1).month

        if month_current != month_next:
            print('rebalance day!')
        # Simply log the closing price of the series from the reference
        self.log('Close 1, %.2f' % self.spy[0])
        self.log('Close 2, %.2f' % self.amzn[0])

        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return

        # Check if we are in the market
        if not self.position:

            # Not yet ... we MIGHT BUY if ...
            if self.spy[0] < self.spy[-1]:
                    # current close less than previous close

                    if self.spy[-1] < self.spy[-2]:
                        # previous close less than the previous close

                        # BUY, BUY, BUY!!! (with default parameters)
                        self.log('BUY 1 CREATE, %.2f' % self.spy[0])
                        self.log('BUY 2 CREATE, %.2f' % self.amzn[0])

                        # Keep track of the created order to avoid a 2nd order
                        self.order = self.order_target_percent(self.datas[0], target = 0.5)
                        self.order_target_percent(self.datas[1], target = 0.5)

        else:

            # Already in the market ... we might sell
            if len(self) >= (self.bar_executed + 5):
                # SELL, SELL, SELL!!! (with all possible default parameters)
                self.log('SELL 1 CREATE, %.2f' % self.spy[0])
                self.log('SELL 2 CREATE, %.2f' % self.amzn[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.order_target_percent(self.datas[0], target = 0)
                self.order_target_percent(self.datas[1], target = 0)

In [40]:
# Create a Data Feed
spy = bt.feeds.YahooFinanceData(
    dataname='SPY',
    # Do not pass values before this date
    fromdate=datetime.datetime(2000, 1, 1),
    # Do not pass values before this date
    todate=datetime.datetime(2000, 12, 31),
    # Do not pass values after this date
    reverse=False)

amzn = bt.feeds.YahooFinanceData(
    dataname='AMZN',
    # Do not pass values before this date
    fromdate=datetime.datetime(2000, 1, 1),
    # Do not pass values before this date
    todate=datetime.datetime(2000, 12, 31),
    # Do not pass values after this date
    reverse=False)

In [62]:
# Create a cerebro entity
cerebro = bt.Cerebro()

# Add a strategy
cerebro.addstrategy(TestStrategy)

# Datas are in a subfolder of the samples. Need to find where the script is
# because it could have been called from anywhere
# modpath = os.path.dirname(os.path.abspath(sys.argv[0]))
# datapath = os.path.join(modpath, '../datas/orcl-1995-2014.txt')
datapath = '/Users/cheng/Google Drive/PhD/Research/Portfolio Selection via TBN/data/orcl-1995-2014.txt'


# Add the Data Feed to Cerebro
cerebro.adddata(spy)
cerebro.adddata(amzn)

# Set our desired cash start
cerebro.broker.setcash(100000.0)

# Set the commission - 0.1% ... divide by 100 to remove the %
cerebro.broker.setcommission(commission=0)

# Print out the starting conditions
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

# Run over everything
cerebro.run()

# Print out the final result
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

-03-07, BUY 1 CREATE, 92.18
2000-03-07, BUY 2 CREATE, 63.50
2000-03-08, BUY EXECUTED, Price: 91.79, Cost: 46996.48, Comm 0.00
2000-03-08, BUY EXECUTED, Price: 63.25, Cost: 46994.75, Comm 0.00
2000-03-08, Close 1, 92.07
2000-03-08, Close 2, 63.62
2000-03-09, Close 1, 94.76
2000-03-09, Close 2, 68.81
2000-03-10, Close 1, 94.25
2000-03-10, Close 2, 66.88
2000-03-13, Close 1, 93.22
2000-03-13, Close 2, 65.31
2000-03-14, Close 1, 91.90
2000-03-14, Close 2, 65.62
2000-03-15, Close 1, 94.04
2000-03-15, Close 2, 63.75
2000-03-15, SELL 1 CREATE, 94.04
2000-03-15, SELL 2 CREATE, 63.75
2000-03-16, SELL EXECUTED, Price: 95.26, Cost: 46996.48, Comm 0.00
2000-03-16, SELL EXECUTED, Price: 62.25, Cost: 46994.75, Comm 0.00
2000-03-16, OPERATION PROFIT, GROSS 1776.64, NET 1776.64
2000-03-16, OPERATION PROFIT, GROSS -743.00, NET -743.00
2000-03-16, Close 1, 98.44
2000-03-16, Close 2, 66.25
2000-03-17, Close 1, 99.09
2000-03-17, Close 2, 64.81
2000-03-20, Close 1, 98.58
2000-03-20, Close 2, 64.19
2000-03-

IndexError: array index out of range

In [48]:
import datetime

x = datetime.datetime(2020, 5, 17)

In [51]:
x.month

5